In [ ]:
import pandas as pd
import matplotlib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, ConvLSTM2D,MaxPooling3D,Flatten
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
IMAGE_SIZE = (124,124)

In [ ]:
# Create the image generatorstrain_datagen = ImageDataGenerator(
train_datagen = ImageDataGenerator(
    rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

# Create the image generators
train_generator = train_datagen.flow_from_directory(
    'Data/train/train',
    target_size=IMAGE_SIZE,
    batch_size=128,
    class_mode='binary')

val_generator = val_datagen.flow_from_directory(
    'Data/test_set/test_set/',  
    target_size=IMAGE_SIZE, 
    batch_size=128,
    class_mode='binary')

In [ ]:
from tweaked_ImageGenerator_v2 import ImageDataGenerator
datagen = ImageDataGenerator()
train_data=datagen.flow_from_directory('Data/train/train', target_size=(128, 128), batch_size=32, frames_per_step=4)
val_data= datagen.flow_from_directory('Data/test_set/test_set', target_size=(128, 128), batch_size=32, frames_per_step=4)

In [ ]:
def img_gen(input_gen,axis):  
    images, labels=next(input_gen) # get the next batch of images and labels
    #images=np.expand_dims(images,axis=axis) # expand dimensions of the images
    yield (images, labels ) #  output a batch of tuples to model.fit
# showing dimension of images have been expanded    
train_x, train_y=next(img_gen(train_data, axis=4))
test_x,test_y = next(img_gen(val_data, axis=4))

In [ ]:
train_x.shape

In [ ]:
rnn = Sequential()
rnn.add(ConvLSTM2D(filters=20, input_shape= (4,128,128, 3),padding = "same",kernel_size=3 , data_format='channels_last'
                       , activation='tanh',return_sequences=True))
rnn.add(Dropout(0.5))
rnn.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
rnn.add(Dropout(0.5))
rnn.add(ConvLSTM2D(filters=20,padding = "same",kernel_size=3 , data_format='channels_last'
                       , activation='tanh',return_sequences=True))
rnn.add(Dropout(0.5))
rnn.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
rnn.add(Flatten())
rnn.add(Dense(2, activation='softmax'))
rnn.summary()
#softmax

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-6, decay=1e-6)
rnn.compile(
    loss='mse',
    optimizer=opt,
    metrics=["accuracy"],
    run_eagerly=True
)

In [ ]:
epochs = 30
batches = 15
total_train_batches = len(train_x)//batches
total_val_batches = len(test_x)//batches

In [ ]:
with tf.device('/GPU:0'):
    history = rnn.fit( x= train_x, y = train_y,
            epochs=30,
            steps_per_epoch = 128,
            validation_data=(test_x,test_y),
            )

In [ ]:
print("[INFO] evaluating network...")
predIdxs = rnn.predict(test_x, batch_size=128)
predIdx = np.round(predIdxs).astype(int)
# show a nicely formatted classification report
print(classification_report(test_y, predIdx,target_names=['fake','real']))

In [ ]:
import keras
from matplotlib import pyplot as plt
%matplotlib inline
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
rnn.save('cnnrnn1.h5')  # creates a HDF5 file 'my_model.h5'